# Make a catalog of SEDs

In [13]:
# Imports
from SEDkit import sed, spectrum, synphot, catalog
import astropy.units as q
import numpy as np
import copy
from pkg_resources import resource_filename
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

First we initialize an empty catalog.

In [14]:
# Make an SED catalog
cat = catalog.SEDCatalog()

Now let's make a few SEDs

In [15]:
# Make an SED
s1 = sed.SED(parallax=(1.7474*q.mas, 0.0501*q.mas), spectral_type='M0V', name='2MASS 261832886')
s1.add_photometry_file(resource_filename('SEDkit', 'data/M0V_photometry.txt'))
s1.make_sed()
cat.add_SED(s1)

Setting age to (<Quantity 6.0 Gyr>, <Quantity 4.0 Gyr>)
Setting radius to (<Quantity 0.8786209573091851 solRad>, <Quantity 0.06782337214316517 solRad>)


In [16]:
# Make 10 other SEDs (I'll just change the parallax)
for i in range(10):
    s2 = copy.copy(s1)
    s2.parallax = abs(np.random.normal(loc=1.7474, scale=0.1))*q.mas, 0.1*q.mas
    s2.name += str(i)
    s2.make_sed()
    cat.add_SED(s2)

Then the (abridged) aggregated results look like this:

In [17]:
cat.results[['name','distance','distance_unc','Lbol_sun','Lbol_sun_unc','Teff','Teff_unc']]

name,distance,distance_unc,Lbol_sun,Lbol_sun_unc,Teff,Teff_unc
,pc,pc,,,K,K
object,object,object,object,object,object,object
2MASS 261832886,572.28,16.41,-1.019,0.025,3424.0,141.0
2MASS 2618328860,570.18,32.51,-1.022,0.05,3418.0,164.0
2MASS 2618328861,601.33,36.16,-0.976,0.052,3510.0,171.0
2MASS 2618328862,595.05,35.41,-0.985,0.052,3492.0,170.0
2MASS 2618328863,561.37,31.51,-1.036,0.049,3391.0,162.0
2MASS 2618328864,590.02,34.81,-0.993,0.051,3477.0,169.0
2MASS 2618328865,634.2,40.22,-0.93,0.055,3605.0,180.0
2MASS 2618328866,561.95,31.58,-1.035,0.049,3393.0,162.0


We can also plot the results by passing the `plot` method two column names.

In [18]:
show(cat.plot('distance', 'Teff'))

Or plot a CMD! You can also pass the difference of two magnitudes as a string like so.

In [19]:
show(cat.plot('2MASS.J-2MASS.Ks', 'M_2MASS.J'))

We can then retrieve the SED for a particular source like so:

In [20]:
my_sed = cat.get_SED('2MASS 261832886')
fig = my_sed.plot()

And save and load from file.

In [13]:
cat.save('/Users/jfilippazzo/Desktop/test.p')

PicklingError: Can't pickle <function ColumnDataSource.<lambda> at 0x112503730>: attribute lookup ColumnDataSource.<lambda> on bokeh.models.sources failed

In [ ]:
new = sed.SEDCatalog().load('/Users/jfilippazzo/Desktop/test.p')

The catalog can also be filtered by providing a column name and value...

In [21]:
cat.filter('name', "'2MASS 2618328861'").results

name,age,age_unc,distance,distance_unc,parallax,parallax_unc,radius,radius_unc,spectral_type,spectral_type_unc,membership,fbol,fbol_unc,mbol,mbol_unc,Lbol,Lbol_unc,Lbol_sun,Lbol_sun_unc,Mbol,Mbol_unc,logg,logg_unc,mass,mass_unc,Teff,Teff_unc,SED,SDSS.u,SDSS.u_unc,M_SDSS.u,M_SDSS.u_unc,SDSS.g,SDSS.g_unc,M_SDSS.g,M_SDSS.g_unc,SDSS.r,SDSS.r_unc,M_SDSS.r,M_SDSS.r_unc,SDSS.i,SDSS.i_unc,M_SDSS.i,M_SDSS.i_unc,SDSS.z,SDSS.z_unc,M_SDSS.z,M_SDSS.z_unc,2MASS.J,2MASS.J_unc,M_2MASS.J,M_2MASS.J_unc,2MASS.H,2MASS.H_unc,M_2MASS.H,M_2MASS.H_unc,2MASS.Ks,2MASS.Ks_unc,M_2MASS.Ks,M_2MASS.Ks_unc,WISE.W1,WISE.W1_unc,M_WISE.W1,M_WISE.W1_unc,WISE.W2,WISE.W2_unc,M_WISE.W2,M_WISE.W2_unc,WISE.W3,WISE.W3_unc,M_WISE.W3,M_WISE.W3_unc
,Myr,Myr,pc,pc,mas,mas,6.957e+08 m,,,,,erg / (cm2 s),erg / (cm2 s),,,erg / s,erg / s,,,,,,,solMass,solMass,K,K,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
2MASS 2618328861,6000.0,4000.0,601.33,36.16,1.662968880585695,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,4.04298477531372e+32,4.877560869570085e+31,-0.976,0.052,7.196,0.066,4.5,0.07,0.8896913720017506,0.0,3510.0,171.0,<SEDkit.sed.SED object at 0x11ae663c8>,20.486,0.073,74076.919,8908.991,17.779,0.006,64288.467,7731.765,16.431,0.004,59414.129,7145.544,15.843,0.004,57287.935,6889.833,15.514,0.005,56098.278,6746.757,14.391,0.029,52037.535,6258.385,13.677,0.032,49455.727,5947.879,13.535,0.034,48942.258,5886.126,13.55,0.027,48996.498,5892.649,13.581,0.035,49108.593,5906.131,12.884,nan,46588.257,nan


...or a wildcard search...

In [22]:
cat.filter('name', "*328861").results

name,age,age_unc,distance,distance_unc,parallax,parallax_unc,radius,radius_unc,spectral_type,spectral_type_unc,membership,fbol,fbol_unc,mbol,mbol_unc,Lbol,Lbol_unc,Lbol_sun,Lbol_sun_unc,Mbol,Mbol_unc,logg,logg_unc,mass,mass_unc,Teff,Teff_unc,SED,SDSS.u,SDSS.u_unc,M_SDSS.u,M_SDSS.u_unc,SDSS.g,SDSS.g_unc,M_SDSS.g,M_SDSS.g_unc,SDSS.r,SDSS.r_unc,M_SDSS.r,M_SDSS.r_unc,SDSS.i,SDSS.i_unc,M_SDSS.i,M_SDSS.i_unc,SDSS.z,SDSS.z_unc,M_SDSS.z,M_SDSS.z_unc,2MASS.J,2MASS.J_unc,M_2MASS.J,M_2MASS.J_unc,2MASS.H,2MASS.H_unc,M_2MASS.H,M_2MASS.H_unc,2MASS.Ks,2MASS.Ks_unc,M_2MASS.Ks,M_2MASS.Ks_unc,WISE.W1,WISE.W1_unc,M_WISE.W1,M_WISE.W1_unc,WISE.W2,WISE.W2_unc,M_WISE.W2,M_WISE.W2_unc,WISE.W3,WISE.W3_unc,M_WISE.W3,M_WISE.W3_unc
,Myr,Myr,pc,pc,mas,mas,6.957e+08 m,,,,,erg / (cm2 s),erg / (cm2 s),,,erg / s,erg / s,,,,,,,solMass,solMass,K,K,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
2MASS 2618328861,6000.0,4000.0,601.33,36.16,1.662968880585695,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,4.04298477531372e+32,4.877560869570085e+31,-0.976,0.052,7.196,0.066,4.5,0.07,0.8896913720017506,0.0,3510.0,171.0,<SEDkit.sed.SED object at 0x11ae663c8>,20.486,0.073,74076.919,8908.991,17.779,0.006,64288.467,7731.765,16.431,0.004,59414.129,7145.544,15.843,0.004,57287.935,6889.833,15.514,0.005,56098.278,6746.757,14.391,0.029,52037.535,6258.385,13.677,0.032,49455.727,5947.879,13.535,0.034,48942.258,5886.126,13.55,0.027,48996.498,5892.649,13.581,0.035,49108.593,5906.131,12.884,nan,46588.257,nan


... or an argument with an operator...

In [23]:
cat.filter('logg', '>=4.5').results

name,age,age_unc,distance,distance_unc,parallax,parallax_unc,radius,radius_unc,spectral_type,spectral_type_unc,membership,fbol,fbol_unc,mbol,mbol_unc,Lbol,Lbol_unc,Lbol_sun,Lbol_sun_unc,Mbol,Mbol_unc,logg,logg_unc,mass,mass_unc,Teff,Teff_unc,SED,SDSS.u,SDSS.u_unc,M_SDSS.u,M_SDSS.u_unc,SDSS.g,SDSS.g_unc,M_SDSS.g,M_SDSS.g_unc,SDSS.r,SDSS.r_unc,M_SDSS.r,M_SDSS.r_unc,SDSS.i,SDSS.i_unc,M_SDSS.i,M_SDSS.i_unc,SDSS.z,SDSS.z_unc,M_SDSS.z,M_SDSS.z_unc,2MASS.J,2MASS.J_unc,M_2MASS.J,M_2MASS.J_unc,2MASS.H,2MASS.H_unc,M_2MASS.H,M_2MASS.H_unc,2MASS.Ks,2MASS.Ks_unc,M_2MASS.Ks,M_2MASS.Ks_unc,WISE.W1,WISE.W1_unc,M_WISE.W1,M_WISE.W1_unc,WISE.W2,WISE.W2_unc,M_WISE.W2,M_WISE.W2_unc,WISE.W3,WISE.W3_unc,M_WISE.W3,M_WISE.W3_unc
,Myr,Myr,pc,pc,mas,mas,6.957e+08 m,,,,,erg / (cm2 s),erg / (cm2 s),,,erg / s,erg / s,,,,,,,solMass,solMass,K,K,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
2MASS 261832886,6000.0,4000.0,572.28,16.41,1.7474,0.0501,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.66179055973561e+32,2.1287295997847215e+31,-1.019,0.025,7.304,0.033,4.5,0.07,0.8896913720017506,0.0,3424.0,141.0,<SEDkit.sed.SED object at 0x11ae6ab00>,20.486,0.073,67092.551,3847.73,17.779,0.006,58227.007,3339.293,16.431,0.004,53812.248,3086.108,15.843,0.004,51886.522,2975.669,15.514,0.005,50809.032,2913.875,14.391,0.029,47131.158,2702.951,13.677,0.032,44792.777,2568.846,13.535,0.034,44327.72,2542.176,13.55,0.027,44376.846,2544.993,13.581,0.035,44478.372,2550.815,12.884,nan,42195.667,nan
2MASS 2618328860,6000.0,4000.0,570.18,32.51,1.753837858963652,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.634965749281282e+32,4.159508308955214e+31,-1.022,0.05,7.312,0.063,4.5,0.07,0.8896913720017506,0.0,3418.0,164.0,<SEDkit.sed.SED object at 0x11ae4b4e0>,20.486,0.073,66601.058,7594.797,17.779,0.006,57800.459,6591.227,16.431,0.004,53418.041,6091.482,15.843,0.004,51506.422,5873.492,15.514,0.005,50436.826,5751.521,14.391,0.029,46785.894,5335.191,13.677,0.032,44464.643,5070.489,13.535,0.034,44002.993,5017.845,13.55,0.027,44051.759,5023.406,13.581,0.035,44152.542,5034.899,12.884,nan,41886.558,nan
2MASS 2618328861,6000.0,4000.0,601.33,36.16,1.662968880585695,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,4.04298477531372e+32,4.877560869570085e+31,-0.976,0.052,7.196,0.066,4.5,0.07,0.8896913720017506,0.0,3510.0,171.0,<SEDkit.sed.SED object at 0x11ae663c8>,20.486,0.073,74076.919,8908.991,17.779,0.006,64288.467,7731.765,16.431,0.004,59414.129,7145.544,15.843,0.004,57287.935,6889.833,15.514,0.005,56098.278,6746.757,14.391,0.029,52037.535,6258.385,13.677,0.032,49455.727,5947.879,13.535,0.034,48942.258,5886.126,13.55,0.027,48996.498,5892.649,13.581,0.035,49108.593,5906.131,12.884,nan,46588.257,nan
2MASS 2618328862,6000.0,4000.0,595.05,35.41,1.6805254642040846,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.958979772294807e+32,4.726823093558116e+31,-0.985,0.052,7.219,0.065,4.5,0.07,0.8896913720017506,0.0,3492.0,170.0,<SEDkit.sed.SED object at 0x11ae56198>,20.486,0.073,72537.751,8633.097,17.779,0.006,62952.684,7492.327,16.431,0.004,58179.625,6924.26,15.843,0.004,56097.608,6676.469,15.514,0.005,54932.67,6537.823,14.391,0.029,50956.301,6064.575,13.677,0.032,48428.137,5763.685,13.535,0.034,47925.33

... or even a tuple of arguments...

In [24]:
cat.filter('M_SDSS.r', ('>50000','<60000')).results

name,age,age_unc,distance,distance_unc,parallax,parallax_unc,radius,radius_unc,spectral_type,spectral_type_unc,membership,fbol,fbol_unc,mbol,mbol_unc,Lbol,Lbol_unc,Lbol_sun,Lbol_sun_unc,Mbol,Mbol_unc,logg,logg_unc,mass,mass_unc,Teff,Teff_unc,SED,SDSS.u,SDSS.u_unc,M_SDSS.u,M_SDSS.u_unc,SDSS.g,SDSS.g_unc,M_SDSS.g,M_SDSS.g_unc,SDSS.r,SDSS.r_unc,M_SDSS.r,M_SDSS.r_unc,SDSS.i,SDSS.i_unc,M_SDSS.i,M_SDSS.i_unc,SDSS.z,SDSS.z_unc,M_SDSS.z,M_SDSS.z_unc,2MASS.J,2MASS.J_unc,M_2MASS.J,M_2MASS.J_unc,2MASS.H,2MASS.H_unc,M_2MASS.H,M_2MASS.H_unc,2MASS.Ks,2MASS.Ks_unc,M_2MASS.Ks,M_2MASS.Ks_unc,WISE.W1,WISE.W1_unc,M_WISE.W1,M_WISE.W1_unc,WISE.W2,WISE.W2_unc,M_WISE.W2,M_WISE.W2_unc,WISE.W3,WISE.W3_unc,M_WISE.W3,M_WISE.W3_unc
,Myr,Myr,pc,pc,mas,mas,6.957e+08 m,,,,,erg / (cm2 s),erg / (cm2 s),,,erg / s,erg / s,,,,,,,solMass,solMass,K,K,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
2MASS 261832886,6000.0,4000.0,572.28,16.41,1.7474,0.0501,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.66179055973561e+32,2.1287295997847215e+31,-1.019,0.025,7.304,0.033,4.5,0.07,0.8896913720017506,0.0,3424.0,141.0,<SEDkit.sed.SED object at 0x11ae6ab00>,20.486,0.073,67092.551,3847.73,17.779,0.006,58227.007,3339.293,16.431,0.004,53812.248,3086.108,15.843,0.004,51886.522,2975.669,15.514,0.005,50809.032,2913.875,14.391,0.029,47131.158,2702.951,13.677,0.032,44792.777,2568.846,13.535,0.034,44327.72,2542.176,13.55,0.027,44376.846,2544.993,13.581,0.035,44478.372,2550.815,12.884,nan,42195.667,nan
2MASS 2618328860,6000.0,4000.0,570.18,32.51,1.753837858963652,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.634965749281282e+32,4.159508308955214e+31,-1.022,0.05,7.312,0.063,4.5,0.07,0.8896913720017506,0.0,3418.0,164.0,<SEDkit.sed.SED object at 0x11ae4b4e0>,20.486,0.073,66601.058,7594.797,17.779,0.006,57800.459,6591.227,16.431,0.004,53418.041,6091.482,15.843,0.004,51506.422,5873.492,15.514,0.005,50436.826,5751.521,14.391,0.029,46785.894,5335.191,13.677,0.032,44464.643,5070.489,13.535,0.034,44002.993,5017.845,13.55,0.027,44051.759,5023.406,13.581,0.035,44152.542,5034.899,12.884,nan,41886.558,nan
2MASS 2618328861,6000.0,4000.0,601.33,36.16,1.662968880585695,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,4.04298477531372e+32,4.877560869570085e+31,-0.976,0.052,7.196,0.066,4.5,0.07,0.8896913720017506,0.0,3510.0,171.0,<SEDkit.sed.SED object at 0x11ae663c8>,20.486,0.073,74076.919,8908.991,17.779,0.006,64288.467,7731.765,16.431,0.004,59414.129,7145.544,15.843,0.004,57287.935,6889.833,15.514,0.005,56098.278,6746.757,14.391,0.029,52037.535,6258.385,13.677,0.032,49455.727,5947.879,13.535,0.034,48942.258,5886.126,13.55,0.027,48996.498,5892.649,13.581,0.035,49108.593,5906.131,12.884,nan,46588.257,nan
2MASS 2618328862,6000.0,4000.0,595.05,35.41,1.6805254642040846,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.958979772294807e+32,4.726823093558116e+31,-0.985,0.052,7.219,0.065,4.5,0.07,0.8896913720017506,0.0,3492.0,170.0,<SEDkit.sed.SED object at 0x11ae56198>,20.486,0.073,72537.751,8633.097,17.779,0.006,62952.684,7492.327,16.431,0.004,58179.625,6924.26,15.843,0.004,56097.608,6676.469,15.514,0.005,54932.67,6537.823,14.391,0.029,50956.301,6064.575,13.677,0.032,48428.137,5763.685,13.535,0.034,47925.33